In [ ]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.7 MB/s eta 0:00:00


In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving forestfires.csv to forestfires.csv


In [ ]:
import io

df = pd.read_csv('forestfires.csv')

In [ ]:
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,small


In [ ]:
df.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,0.143133,0.162476,0.183752,0.117988,0.123791,0.104449,0.017408,0.355899,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,0.350548,0.369244,0.387657,0.322907,0.329662,0.306138,0.130913,0.479249,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.isnull().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [ ]:
df.duplicated().sum()

8

In [ ]:
df.drop_duplicates(keep='first', inplace = True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.shape

(509, 31)

In [ ]:
pd.set_option('display.max_columns', None)
df.corr()

<ipython-input-53-c2592685dc60>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
FFMC,1.000000,0.382925,0.331956,0.531926,0.431744,-0.306262,-0.030409,0.052280,0.040998,0.020889,-0.057581,-0.021744,-0.088579,0.073127,0.007268,0.094854,-0.116738,0.225720,-0.136614,-0.281054,-0.455128,0.033137,-0.041972,-0.074706,-0.036983,-0.088879,-0.005172,0.080444
DMC,0.382925,1.000000,0.681446,0.309459,0.470875,0.062772,-0.104563,0.071017,0.076932,-0.011152,-0.105205,-0.003147,0.028050,0.079932,-0.002803,0.021687,-0.198536,0.490160,-0.177044,-0.319691,-0.106256,0.000617,-0.047759,-0.404974,-0.082312,-0.074642,-0.188126,0.119779
DC,0.331956,0.681446,1.000000,0.229757,0.496608,-0.047971,-0.202856,0.033772,0.051542,-0.001642,-0.053198,-0.029112,-0.002249,0.047835,0.026066,0.022670,-0.270129,0.270894,-0.106347,-0.402201,-0.115871,-0.101518,-0.178719,-0.648890,-0.115010,-0.078927,0.094091,0.538299
ISI,0.531926,0.309459,0.229757,1.000000,0.395858,-0.135955,0.104834,0.062132,0.008429,0.051599,-0.159333,-0.037140,-0.005213,-0.019173,0.062274,0.125896,-0.106785,0.337101,-0.162761,-0.250494,-0.103846,0.020932,0.119765,-0.143589,-0.060655,-0.076748,-0.071401,-0.069600
temp,0.431744,0.470875,0.496608,0.395858,1.000000,-0.532503,-0.227754,0.067911,0.099173,-0.077766,-0.135289,0.038863,0.015661,0.051864,0.034738,0.091309,-0.156825,0.349583,-0.329774,-0.319901,-0.146547,0.143987,0.052779,-0.342129,-0.045372,-0.053720,-0.052918,0.091428
RH,-0.306262,0.062772,-0.047971,-0.135955,-0.532503,1.000000,0.070721,0.094343,-0.074554,0.069599,0.010311,-0.024118,0.140341,-0.128270,-0.021152,-0.087122,0.021627,0.046842,-0.047670,0.141593,0.171900,0.013808,0.014425,-0.083581,0.087384,-0.035967,-0.072335,-0.061631
wind,-0.030409,-0.104563,-0.202856,0.104834,-0.227754,0.070721,1.000000,0.059332,0.012623,0.121056,-0.062712,-0.068421,0.025971,-0.055820,0.052331,-0.022181,0.048864,0.027413,0.270919,-0.028820,-0.070217,-0.039880,0.010124,0.181610,0.015311,0.012049,-0.053404,-0.180211
rain,0.052280,0.071017,0.033772,0.062132,0.067911,0.094343,0.059332,1.000000,-0.006511,-0.001723,-0.028473,-0.030252,-0.015749,-0.025236,0.127243,-0.019189,-0.009262,0.088572,-0.009262,-0.013961,-0.004336,-0.012411,-0.012437,-0.019189,-0.004336,-0.003063,-0.012030,-0.049319
area,0.040998,0.076932,0.051542,0.008429,0.099173,-0.074554,0.012623,-0.006511,1.000000,-0.052390,-0.022178,0.087647,-0.020657,0.020685,-0.001002,-0.011046,-0.008589,-0.000948,0.000706,-0.021205,-0.012737,0.005566,-0.019083,-0.048394,0.006124,-0.008997,-0.017284,0.055191
dayfri,0.020889,-0.011152,-0.001642,0.051599,-0.077766,0.069599,0.121056,-0.001723,-0.052390,1.000000,-0.182056,-0.193432,-0.210075,-0.161357,-0.165896,-0.150484,-0.018868,-0.100652,-0.018868,0.047592,-0.027723,-0.048598,-0.018562,0.041050,0.057284,-0.019584,-0.045470,0.111903


In [ ]:
df.drop(columns = ['month','day'], inplace = True)

In [ ]:
y = [0  if i == 'small' else 1 for i in df.size_category ]

In [ ]:
x = df.drop(columns = ['size_category'])


In [ ]:
sc = StandardScaler()
x = sc.fit_transform(x)
x

array([[-7.96039810e-01, -1.31838569e+00, -1.82893094e+00, ...,
        -4.43678255e-02, -1.74253759e-01, -7.14412875e-01],
       [-3.25665522e-03, -1.17405250e+00,  4.89256926e-01, ...,
        -4.43678255e-02,  5.73875712e+00, -7.14412875e-01],
       [-3.25665522e-03, -1.04383887e+00,  5.61044929e-01, ...,
        -4.43678255e-02,  5.73875712e+00, -7.14412875e-01],
       ...,
       [-1.62485856e+00, -8.39889805e-01,  4.75141308e-01, ...,
        -4.43678255e-02, -1.74253759e-01, -7.14412875e-01],
       [ 6.81419706e-01,  5.61083388e-01,  2.69859884e-01, ...,
        -4.43678255e-02, -1.74253759e-01, -7.14412875e-01],
       [-2.00323234e+00, -1.68235633e+00, -1.77892132e+00, ...,
         2.25388553e+01, -1.74253759e-01, -7.14412875e-01]])

In [ ]:
y = np.array(y)

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.20, random_state=53)

In [ ]:
pd.Series(y).value_counts()

0    371
1    138
dtype: int64

In [ ]:
ann = Sequential()

ann.add(Dense(units=10,activation='relu'))
ann.add(Dense(units=1, activation = 'sigmoid'))

ann.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = 'accuracy')

ann.fit(xtrain, ytrain, epochs = 100)

Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 0.8779 - accuracy: 0.4963
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 0.8224 - accuracy: 0.5135
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7795 - accuracy: 0.5356
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7400 - accuracy: 0.5700
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7073 - accuracy: 0.6044
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.6815 - accuracy: 0.6364
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: 0.6593 - accuracy: 0.6658
Epoch 8/100
13/13 [==============================] - 0s 1ms/step - loss: 0.6421 - accuracy: 0.6732
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: 0.6270 - accuracy: 0.6806
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: 0.6132 - accuracy: 0.6904
Epoch 11/

In [ ]:
ypred = ann.predict(xtest)
ypred

4/4 [==============================] - 0s 2ms/step


array([[0.10066215],
       [0.03231611],
       [0.9998364 ],
       [0.15446332],
       [0.49746743],
       [0.9883137 ],
       [0.15556191],
       [0.890742  ],
       [0.08674853],
       [0.20282426],
       [0.21218356],
       [0.06648163],
       [0.18031356],
       [0.22608034],
       [0.0726259 ],
       [0.5556343 ],
       [0.15009816],
       [0.1496028 ],
       [0.22619012],
       [0.04828377],
       [0.06804367],
       [0.54414403],
       [0.5354025 ],
       [0.20279236],
       [0.23492116],
       [0.9998556 ],
       [0.18067807],
       [0.052683  ],
       [0.31523195],
       [0.18218675],
       [0.26736486],
       [0.17737897],
       [0.1584325 ],
       [0.20947322],
       [0.98843104],
       [0.46682498],
       [0.17342287],
       [0.02303278],
       [0.16546482],
       [0.10477816],
       [0.8690385 ],
       [0.335645  ],
       [0.36946765],
       [0.2723352 ],
       [0.20082685],
       [0.18260595],
       [0.14888516],
       [0.199

In [ ]:
ypred = ypred > 0.5
ypred

array([[False],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [

In [ ]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91        75
           1       0.84      0.59      0.70        27

    accuracy                           0.86       102
   macro avg       0.85      0.78      0.80       102
weighted avg       0.86      0.86      0.85       102



In [ ]:
ann.evaluate(xtrain, ytrain)

13/13 [==============================] - 0s 1ms/step - loss: 0.2949 - accuracy: 0.8845


[0.29488784074783325, 0.8845208883285522]

In [ ]:
ann.evaluate(xtest, ytest)

4/4 [==============================] - 0s 2ms/step - loss: 0.4198 - accuracy: 0.8627


[0.41982582211494446, 0.8627451062202454]

In [ ]:
xtrain.shape

(407, 28)

In [ ]:
def hyper(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=256, step=8),
                      activation = hp.Choice('activation'+str(i), values= ['relu','tanh','sigmoid']),
                      input_dim = 28))
  else:
    model.add(Dense(hp.Int('units'+str(i), min_value=8,max_value=256, step=8),
                    activation = hp.Choice('activation'+str(i), values = ['relu','tanh','sigmoid'])))
  counter+=1

  model.add(Dense(units=1, activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy', metrics = 'accuracy')
  return model

In [ ]:
tuner = kf.RandomSearch(hyper, objective = 'val_accuracy',
                        max_trials = 3)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(xtrain,ytrain, epochs = 5, validation_data= (xtest,ytest))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 128,
 'activation0': 'relu',
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'tanh',
 'units2': 88,
 'activation2': 'tanh',
 'units3': 80,
 'activation3': 'sigmoid',
 'units4': 32,
 'activation4': 'sigmoid',
 'units5': 96,
 'activation5': 'tanh',
 'units6': 56,
 'activation6': 'tanh',
 'units7': 80,
 'activation7': 'tanh'}

In [ ]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tuned_model.fit(xtrain,ytrain, epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
13/13 [==============================] - 1s 14ms/step - loss: 0.6273 - accuracy: 0.7273 - val_loss: 0.6100 - val_accuracy: 0.7353
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 0.6070 - accuracy: 0.7273 - val_loss: 0.5952 - val_accuracy: 0.7353
Epoch 3/50
13/13 [==============================] - 0s 5ms/step - loss: 0.5969 - accuracy: 0.7273 - val_loss: 0.5873 - val_accuracy: 0.7353
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5919 - accuracy: 0.7273 - val_loss: 0.5835 - val_accuracy: 0.7353
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5890 - accuracy: 0.7273 - val_loss: 0.5812 - val_accuracy: 0.7353
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5876 - accuracy: 0.7273 - val_loss: 0.5799 - val_accuracy: 0.7353
Epoch 7/50
13/13 [==============================] - 0s 5ms/step - loss: 0.5869 - accuracy: 0.7273 - val_loss: 0.5792 - val_accuracy: 0.7353
Epoch 8/50
13/13 [=

In [ ]:
tuned_model.evaluate(xtrain,ytrain)

13/13 [==============================] - 0s 1ms/step - loss: 0.5860 - accuracy: 0.7273


[0.5859635472297668, 0.7272727489471436]

In [ ]:
tuned_model.evaluate(xtest, ytest)

4/4 [==============================] - 0s 2ms/step - loss: 0.5783 - accuracy: 0.7353


[0.5782942175865173, 0.7352941036224365]

In [134]:
def hyper1(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=256, step=5),
                      activation = hp.Choice('activation'+str(i), values= ['relu','tanh','sigmoid']),
                      input_dim = 28))
  else:
    model.add(Dense(hp.Int('units'+str(i), min_value=8,max_value=256, step=5),
                    activation = hp.Choice('activation'+str(i), values = ['relu','tanh','sigmoid'])))
  counter+=1

  model.add(Dense(units=1, activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy', metrics = 'accuracy')
  return model

In [135]:
tuner1 = kf.RandomSearch(hyper1, objective = 'val_accuracy',
                        max_trials = 3)

Reloading Tuner from ./untitled_project/tuner0.json


In [136]:
tuner1.search(xtrain,ytrain, epochs = 5, validation_data= (xtest,ytest))

In [137]:
tuned_model1 = tuner1.get_best_models(num_models=1)[0]

In [138]:
tuned_model1.fit(xtrain,ytrain, epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
13/13 [==============================] - 3s 46ms/step - loss: 0.6275 - accuracy: 0.7273 - val_loss: 0.6102 - val_accuracy: 0.7353
Epoch 2/50
13/13 [==============================] - 0s 7ms/step - loss: 0.6072 - accuracy: 0.7273 - val_loss: 0.5950 - val_accuracy: 0.7353
Epoch 3/50
13/13 [==============================] - 0s 7ms/step - loss: 0.5968 - accuracy: 0.7273 - val_loss: 0.5870 - val_accuracy: 0.7353
Epoch 4/50
13/13 [==============================] - 0s 7ms/step - loss: 0.5914 - accuracy: 0.7273 - val_loss: 0.5830 - val_accuracy: 0.7353
Epoch 5/50
13/13 [==============================] - 0s 5ms/step - loss: 0.5887 - accuracy: 0.7273 - val_loss: 0.5809 - val_accuracy: 0.7353
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 0.5874 - accuracy: 0.7273 - val_loss: 0.5799 - val_accuracy: 0.7353
Epoch 7/50
13/13 [==============================] - 0s 7ms/step - loss: 0.5870 - accuracy: 0.7273 - val_loss: 0.5793 - val_accuracy: 0.7353
Epoch 8/50
13/13 [=

In [151]:
def hyper1(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units'+str(i),  min_value=8, max_value=128),
                      activation = hp.Choice('activation'+str(i), values= ['relu','tanh','sigmoid']),
                      input_dim = 28))
  else:
    model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128),
                    activation = hp.Choice('activation'+str(i), values = ['relu','tanh','sigmoid'])))
  counter+=1

  model.add(Dense(units=1, activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy', metrics = 'accuracy')
  return model

In [152]:
tuner1 = kf.RandomSearch(hyper1, objective = 'val_accuracy',
                        max_trials = 3)

Reloading Tuner from ./untitled_project/tuner0.json


In [153]:
tuner1.search(xtrain,ytrain, epochs = 5, validation_data= (xtest,ytest))

In [154]:
tuned_model1 = tuner1.get_best_models(num_models=1)[0]

In [155]:
tuned_model1.fit(xtrain,ytrain, epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
13/13 [==============================] - 1s 14ms/step - loss: 0.6276 - accuracy: 0.7273 - val_loss: 0.6102 - val_accuracy: 0.7353
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6070 - accuracy: 0.7273 - val_loss: 0.5952 - val_accuracy: 0.7353
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5967 - accuracy: 0.7273 - val_loss: 0.5876 - val_accuracy: 0.7353
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5919 - accuracy: 0.7273 - val_loss: 0.5833 - val_accuracy: 0.7353
Epoch 5/50
13/13 [==============================] - 0s 5ms/step - loss: 0.5892 - accuracy: 0.7273 - val_loss: 0.5814 - val_accuracy: 0.7353
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5878 - accuracy: 0.7273 - val_loss: 0.5802 - val_accuracy: 0.7353
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5872 - accuracy: 0.7273 - val_loss: 0.5797 - val_accuracy: 0.7353
Epoch 8/50
13/13 [=